### 子类化内置类型

In [1]:
# 一个不允许重复值的字典
class DistinctError(ValueError):
    """如果向distinctdict添加重复值，则引发这个错误"""
    ...

class DistinctDict(dict):
    """不接受重复值的字典"""
    def __setitem__(self, key, value):
        if value in self.values():
            if (
                    (key in self and self[key] != value) or key not in self
            ):
                raise DistinctError("this value already exists for different key")

        super().__setitem__(key, value)

my = DistinctDict()
my['key'] = 'value'
my['other_key'] = 'value'

DistinctError: this value already exists for different key

In [2]:
my['other_key'] = 'value2'
my

{'key': 'value', 'other_key': 'value2'}

### 访问超类中的方法

In [3]:
# 访问超类中的方法
# 旧的写法
class Mama:
    def says(self):
        print('do your homework')


class Sister(Mama):
    def says(self):
        Mama.says(self)
        print('and clean your bedroom')

Sister().says()

do your homework
and clean your bedroom


In [4]:
# super方法访问超类
class Sister(Mama):
    def says(self):
        super().says() # 简化形式可以在方法内部使用
        print('and clean your bedroom')

Sister().says()

do your homework
and clean your bedroom


In [5]:
# super不在方法内部使用时
anita = Sister()
super(anita.__class__, anita).says()

do your homework


In [13]:
# 如果super只提供了一个参数，那么它将返回一个未绑定（unbound）类型
# 这一点在与classmethod一起使用时特别有用
class Pizza:
    def __init__(self, toppings):
        self.toppings = toppings

    def __repr__(self):
        return 'Pizza with ' + ' and '.join(self.toppings)

    @classmethod
    def recommend(cls):
        """推荐任意馅料（toppings）的某种披萨"""
        return cls(['spam', 'ham', 'eggs'])


class VikingPizza(Pizza):
    @classmethod
    def recommend(cls):
        """推荐与super相同的内容，但多加了午餐肉（spam）"""
        recommended = super().recommend()
        recommended.toppings += ['spam'] * 5
        return recommended

print(Pizza.recommend())
print(VikingPizza.recommend())

Pizza with spam and ham and eggs
Pizza with spam and ham and eggs and spam and spam and spam and spam and spam


In [20]:
# 错误案例 - 混用super与显式类调用
class A:
    def __init__(self):
        print('A', end=' ')
        super().__init__()


class B:
    def __init__(self):
        print('B', end=' ')
        super().__init__()


class C(A, B):
    def __init__(self):
        print('C', end=' ')
        A.__init__(self)
        B.__init__(self)


# 类的继承顺序
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

In [21]:
# C的实例调用了A.__init__(self) -> super(A, self).__init__() —> B.__init__()方法
# 因此会打印2次B
C()

C A B B 

In [22]:
# 初始化过程中的参数传递
class CommonBase:
    def __init__(self):
        print('CommonBase')
        super().__init__()


class Base1(CommonBase):
    def __init__(self):
        print('Base1')
        super().__init__()


class Base2(CommonBase):
    def __init__(self, arg):
        print('Base2')
        super().__init__()


class MyClass(Base1, Base2):
    def __init__(self, arg):
        print('my base')
        super().__init__(arg)

MyClass(10)

my base


TypeError: Base1.__init__() takes 1 positional argument but 2 were given

### MRO

In [16]:
# MRO：方法解析顺序
class CommonBase:
    def method(self):
        print('CommonBase')


class Base1(CommonBase):
    pass


class Base2(CommonBase):
    def method(self):
        print('Base2')


class MyClass(Base1, Base2):
    pass


# 调用类中方法的顺序
MyClass.__mro__

(__main__.MyClass, __main__.Base1, __main__.Base2, __main__.CommonBase, object)

In [17]:
MyClass().method()

Base2


### 高级属性访问模式

In [23]:
# 名称修饰：以__开始的变量是类的私有属性
# 直接访问私有属性会报错，该属性会被解释器重命名
class MyClass:
    __secret_value = 1


instance_of = MyClass()
instance_of.__secret_value

AttributeError: 'MyClass' object has no attribute '__secret_value'

In [24]:
dir(MyClass)

['_MyClass__secret_value',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [25]:
instance_of._MyClass__secret_value

1

In [26]:
# 描述符允许你自定义在引用一个对象的属性时应该完成的事
# 描述符是Python中复杂属性访问的基础
# 它基于3个特殊方法，这3个方法组成了描述符协议：
# 1.__set__(self, instance, value)：在设置属性时将调用这一方法
# 2.__get__(self, instance, owner)：在读取属性时将调用这一方法
# 3.__delete__(self, obj)：对属性调用del时将调用这一方法
# 实现了__get__()和__set__()的描述符被称为数据描述符
class RevealAccess:
    """一个数据描述符，正常设定值并返回值，同时打印出记录访问的信息"""
    def __init__(self, initval=None, name='var'):
        self.value = initval
        self.name = name

    def __get__(self, instance, owner):
        print('Retrieving', self.name)
        return self.value

    def __set__(self, instance, value):
        print('Updating', self.name)
        self.value = value


class MyClass:
    x = RevealAccess(10, 'var x')
    y = 5

m = MyClass()
m.x

Retrieving var x


10

In [27]:
m.x = 20

Updating var x


In [28]:
m.x

Retrieving var x


20

In [29]:
m.y

5

In [30]:
# 描述符的一个示例：延迟求值属性
class InitOnAccess:
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        self._initialized = None

    def __get__(self, instance, owner):
        if self._initialized is None:
            print('initialized!')
            self._initialized = self.klass(*self.args, **self.kwargs)
        else:
            print('cached')
        return self._initialized


class MyClass:
    lazily_initialized = InitOnAccess(list, 'argument')

m = MyClass
m.lazily_initialized

initialized!


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']

In [31]:
m.lazily_initialized

cached


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']